# Watching SVG
[Inkscape SVG](https://inkscape.org) files are a great way to quickly prototype visualizations and diagrams, and can also store arbitrary XML data like RDF. With `ipywatchdog`, you can automatically update the state of an SVG based on saves in Inkscape, then continue to manipulate the data with python libraries like [`PyQuery`](https://pythonhosted.org/pyquery) or JS libraries like [`d3`](https://d3js.org).

In [9]:
from ipywidgets import widgets
from ipywatchdogwidget import Watchdog

The [HTML Widget](https://nbviewer.jupyter.org/github/ipython/ipywidgets/blob/master/examples/notebooks/Widget%20List.ipynb#HTML) can render SVG elements just fine.

In [10]:
html = widgets.HTML(value="""<svg><text x="20" y="20">This is not the file</text></svg>""")

Let's create a `Watchdog` to watch any SVG file in this directory...

In [11]:
dog = Watchdog(path=".", file="*.svg")
dog.ls()

['./drawing.svg']

and update that `HTML` based on file events that we see....

In [12]:
@dog.on_any
def show_svg(self, content):
    with open(content["src_path"]) as fp:
        html.value = fp.read()

In [13]:
dog.start()
html

Now, if there aren't any changes yet, we can force an update...

In [14]:
dog.touch()

Since the `HTML` widget makes the DOM immediately available, we can mix live effects with live reloading.

In [15]:
%%javascript
require(["//cdn.jsdelivr.net/g/d3js"], function(d3){
    setInterval(function(){
        var t = +(new Date());
        var svg = d3.select("svg")
        
        svg.selectAll("tspan")
           .transition()
           .style({fill: d3.hsl(t % 256, 0.5, 0.5)});
    }, 1000)
});

<IPython.core.display.Javascript object>

If you're done watching, you can stop the `Watchdog`.

In [16]:
dog.stop()